In [51]:
# Jupyter notebook needs this or else it will crash
import datetime
import nest_asyncio
nest_asyncio.apply()

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import P # NEW!!! Import predicates (gt, gte, lt, lte, etc.)
from gremlin_python.process.traversal import Cardinality # NEW!!! Import Cardinality such as list_, set_ and single.
from gremlin_python.driver.protocol import GremlinServerError # Gremlin server error
from gremlin_python.process.traversal import Pop # for Pop.all_ in select(Pop.all_, 'v')

# Instantiate a Gremlin Graph
graph = Graph()

# Connect to the server, instantiate traversal of graph.
g = graph.traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

In [52]:
def connected(name1: str, name2: str, time: float) -> bool:
    """
    Given two vertices labelled with <name1> and <name2>, determine whether they were connected at time <time>. 
    Do so by sending a Gremlin query to determine whether there exists an edge between the 
    two vertices such that <time> falls in between their "start_time" and "end_time" parameters.
    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    # Get the vertices associated with the names
    # v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

    # Return whether there are edges that:
    #   - connect v1 and v2, 
    #   - labelled 'connection',
    #   - have a start time that is less than or equal to <time>
    #   - either do not have an end time or have an end time that is greater than or equal to <time>
    return g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
            __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
            __.or_(
                __.hasNot('end'),           # end time doesn't have to exist 
                __.has('end', P.gt(time))  # OR end time must be greater than <time>
            )
        ).count().next() > 0

In [53]:
def set_connection(name1: str, name2: str, time: float, connection: bool) -> None:
    """
    Given two vertices labelled with <name1> and <name2>, create a new connection or terminate their existing connection, based on the value of <bool>. Label with time <time>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    success = False

    while not success:

        try:

            if connection:
                # Add an edge labelled 'connection' with a start time of <time>
                g.V().has('name', name1).as_("a").not_( # NEGATE 
                    __.bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
                        __.has('start', P.lte(time)),
                        __.or_(
                            __.hasNot('end'),
                            __.has('end', P.gt(time))
                        )
                    )
                ).V().has('name', name2).as_("b").addE('connection').from_("a").to("b").property('start', time).iterate()

            else:
                # For all edges between v1 and v2 labelled 'connection' (there should only be one) that do not have an 'end' property, create an end property of <time>.
                g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').hasNot('end').property('end', time).iterate()
        
            success = True

        except GremlinServerError as e:
            print(f"{datetime.now().strftime('%H:%M:%S')} ERROR: Failed to set {connection} connection between {name1} and {name2} at {time}", e)

In [54]:
def add_component(name: str) -> None:
    """
    Add a vertex to the graph with a 'name' property of <name>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    success = False

    while not success:
        try:
            g.addV().property('name', name).next()
            success = True
        except GremlinServerError as e:
            print(f"{datetime.now().strftime('%H:%M:%S')} ERROR: Failed to add component of name {name}", e)


In [55]:
def add_type(type: str) -> None:
    """
    Add a vertex to the graph with a 'type' property of <type>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    success = False

    while not success:
        try:
            g.addV().property('name', type).next()
            success = True
        except GremlinServerError as e:
            print(f"{datetime.now().strftime('%H:%M:%S')} ERROR: Failed to add type vertex of name {type}", e)


def set_type(name: str, type: str) -> None:
    """
    Connect the vertex labelled with <name> to a vertex labelled <type> with an edge going into the <type> vertex labelled with "type".

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    success = False

    while not success:
        try:
            g.V().has('name', name).as_("a").V().has('name', type).as_("b").addE("type").from_("a").to("b").next()
            success = True
        except GremlinServerError as e:
            print(f"{datetime.now().strftime('%H:%M:%S')} ERROR: Failed to set type of {name} to {type}", e)

In [56]:
# This will put it in V-E-V-E-V-...-V form as a list per path.
def find_paths(name1: str, name2: str, avoid_type: str, time: float):
    """
    Given two vertices labelled with <name1> and <name2>, return the paths that connect the vertices by edges that were active at <time> as a list.

    Avoid vertices of type avoid_type.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """
    while True:
        try:
            return g.V().has('name', name1).repeat(
            __.bothE('connection').and_(
                __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
                __.or_(
                    __.hasNot('end'),           # end time doesn't have to exist 
                    __.has('end', P.gt(time))  # OR end time must be greater than <time>
                )
            ).otherV().not_(__.outE('type').inV().has('name', avoid_type)).simplePath()
        ).until(__.has('name', name2)).path().toList()
        except GremlinServerError as e:
            print(f"{datetime.now().strftime('%H:%M:%S')} ERROR: Could not find paths between {name1} and {name2} at time {time} avoiding {avoid_type}", e)
    

In [57]:
# Clear vertices
g.V().drop().iterate()
g.E().drop().iterate()

"""
(Temporary) Naming scheme:
 - COR######: Correlator input
 - ANT######: Antenna
 - DPF######: Dual-Polarization Feed
 - BLN######: (Active) Balun
 - RFT######: RFoF transmitter
 - OPF######: Optical Fiber
 - RFR######: RFoF receiver
 - ADC######: Analog-to-Digital converter
"""

# Set up the types
types = ['COR', 'ANT', 'DPF', 'BLN', 'RFT', 'OPF', 'RFR', 'ADC']
for type in types:
    add_type(type)


# Correlator node name
cor = 'COR000000'

# Add a correlator input node
add_component(cor)
set_type(cor, 'COR')

# number of HIRAX dishes
dishes = 512

# Connections at every 3.
max_time = 30
connections = [(i, bool(i % 2)) for i in range(0, max_time // 3 + 1, 3)]


# Add the components and connect them at different times.
for i in range(1, dishes + 1):



    # The names of the components to refer to
    ant = f'ANT{str(i).zfill(6)}'
    dpf = f'DPF{str(i).zfill(6)}'
    bln = (f'BLN{str(2 * i - 1).zfill(6)}', f'BLN{str(2 * i).zfill(6)}')
    rft = (f'RFT{str(2 * i - 1).zfill(6)}', f'RFT{str(2 * i).zfill(6)}')
    opf = (f'OPF{str(2 * i - 1).zfill(6)}', f'OPF{str(2 * i).zfill(6)}')
    rfr = (f'RFR{str(2 * i - 1).zfill(6)}', f'RFR{str(2 * i).zfill(6)}')
    adc = (f'ADC{str(2 * i - 1).zfill(6)}', f'ADC{str(2 * i).zfill(6)}')

    now = datetime.datetime.now()

    add_component(ant)
    add_component(dpf)

    set_type(ant, 'ANT')
    set_type(dpf, 'DPF')

    for ind in (0, 1):
        add_component(bln[ind])
        add_component(rft[ind])
        add_component(opf[ind])
        add_component(rfr[ind])
        add_component(adc[ind])

        set_type(bln[ind], 'BLN')
        set_type(rft[ind], 'RFT')
        set_type(opf[ind], 'OPF')
        set_type(rfr[ind], 'RFR')
        set_type(adc[ind], 'ADC')

    for (time, connection) in connections:

        set_connection(name1=ant, name2=dpf, time=time, connection=connection)

        for ind in (0, 1):

            # Pairs of names to connect
            pairs = [(ant, dpf), (dpf, bln[ind]), (bln[ind], rft[ind]), (rft[ind], opf[ind]), (opf[ind], rfr[ind]), (rfr[ind], adc[ind]), (adc[ind], cor)]

            for pair in pairs:
                set_connection(name1=pair[0], name2=pair[1], time=time, connection=connection)

    print(f"Dish {i} done, took {(datetime.datetime.now() - now).total_seconds()}")



# EXPORT THE GRAPH.
# g.io("path_benchmark_small_graph.xml").write().next()




Dish 1 done, took 0.648949
Dish 2 done, took 0.708764
Dish 3 done, took 0.718444
Dish 4 done, took 0.678165
Dish 5 done, took 0.692349
Dish 6 done, took 0.696772
Dish 7 done, took 0.727032
Dish 8 done, took 0.768483
Dish 9 done, took 0.763939
Dish 10 done, took 0.781187
Dish 11 done, took 0.789523
Dish 12 done, took 0.796669
Dish 13 done, took 0.812057
Dish 14 done, took 0.811168
Dish 15 done, took 0.814916
Dish 16 done, took 0.882981
Dish 17 done, took 0.886572
Dish 18 done, took 0.927671
Dish 19 done, took 0.927374
Dish 20 done, took 0.940228
Dish 21 done, took 0.936556
Dish 22 done, took 0.930461
Dish 23 done, took 0.955
Dish 24 done, took 1.013655
Dish 25 done, took 1.042756
Dish 26 done, took 1.037551
Dish 27 done, took 1.039735
Dish 28 done, took 1.098501
Dish 29 done, took 1.057816
Dish 30 done, took 1.078663
Dish 31 done, took 1.087522
Dish 32 done, took 1.084984
Dish 33 done, took 1.182546
Dish 34 done, took 1.15151
Dish 35 done, took 1.169337
Dish 36 done, took 1.186616
Dish 

In [58]:
# To benchmark, try finding all of the paths from the correlator to each of the antennas, and time it.
iterations = 1

times_to_query = [1]

total = 0

print(f"Dishes: {dishes}")
print(f"Times to check: {times_to_query}")
print(f"Iterations: {iterations}")

for _ in range(iterations):
    for i in range(len(times_to_query)):
        time = times_to_query[i]
        for dish in range(dishes):

            success = False
            while not success:
                try:
                    then = datetime.datetime.now()
                    find_paths(name1=cor, name2=f'ANT{str(dish).zfill(6)}', time=time, avoid_type='')

                    total += (datetime.datetime.now() - then).total_seconds()

                    print(f"Dish {dish} done")

                    success = True
                except GremlinServerError:
                    print("Uh oh! Backend exception!")

        print(f"{len(times_to_query) - i - 1} times left to check!")

print("Total time:", total)
print("Average time per call:", total / (iterations * len(times_to_query) * dishes))
        

Dishes: 512
Times to check: [1]
Iterations: 1
Dish 0 done
Dish 1 done
Dish 2 done
Dish 3 done
Dish 4 done
Dish 5 done
Dish 6 done
Dish 7 done
Dish 8 done
Dish 9 done
Dish 10 done
Dish 11 done
Dish 12 done
Dish 13 done
Dish 14 done
Dish 15 done
Dish 16 done
Dish 17 done
Dish 18 done
Dish 19 done
Dish 20 done
Dish 21 done
Dish 22 done
Dish 23 done
Dish 24 done
Dish 25 done
Dish 26 done
Dish 27 done
Dish 28 done
Dish 29 done
Dish 30 done
Dish 31 done
Dish 32 done
Dish 33 done
Dish 34 done
Dish 35 done
Dish 36 done
Dish 37 done
Dish 38 done
Dish 39 done
Dish 40 done
Dish 41 done
Dish 42 done
Dish 43 done
Dish 44 done
Dish 45 done
Dish 46 done
Dish 47 done
Dish 48 done
Dish 49 done
Dish 50 done
Dish 51 done
Dish 52 done
Dish 53 done
Dish 54 done
Dish 55 done
Dish 56 done
Dish 57 done
Dish 58 done
Dish 59 done
Dish 60 done
Dish 61 done
Dish 62 done
Dish 63 done
Dish 64 done
Dish 65 done
Dish 66 done
Dish 67 done
Dish 68 done
Dish 69 done
Dish 70 done
Dish 71 done
Dish 72 done
Dish 73 done
Di